%undefined
# Lakeflow Jobs: Orchestrate Your Pipelines 🚀

Welcome to the **Lakeflow Jobs** demo! This notebook teaches you how to create, configure, and manage jobs to orchestrate your Lakeflow pipelines.

---

## What you'll learn:

* 📚 **Part 1:** Understanding Lakeflow Jobs concepts
* 🔧 **Part 2:** Creating jobs programmatically with the Jobs API
* ⚙️ **Part 3:** Job configuration and parameters
* ⏰ **Part 4:** Scheduling and triggers
* 📊 **Part 5:** Monitoring and notifications
* ✅ **Part 6:** Best practices and patterns

---

## Prerequisites:

* Complete [Lakeflow Pipeline Fundamentals](#notebook/2846436383063456)
* Complete [Lakeflow Expectations](#notebook/2846436383063443)
* Complete [Lakeflow Auto CDC](#notebook/2846436383063462)
* Basic understanding of Delta Live Tables
* Familiarity with Python

---

**Let's get started!** 🎯

%undefined
## Part 1: Understanding Lakeflow Jobs 📚

Before creating jobs, let's understand what they are and why they matter.

---

%undefined
### 🎯 What are Lakeflow Jobs?

**Definition:**
* Jobs are **orchestration units** that run your Lakeflow pipelines
* Automate pipeline execution on a schedule or trigger
* Manage compute resources and configurations
* Monitor execution and send notifications
* Part of Databricks Workflows

---

### **The Problem Without Jobs:**

**Manual pipeline execution:**
* ❌ Must manually click "Start" in the UI
* ❌ No automation or scheduling
* ❌ Hard to coordinate multiple pipelines
* ❌ No centralized monitoring
* ❌ Difficult to manage in production

---

### **The Solution With Jobs:**

**Automated orchestration:**
* ✅ Schedule pipelines to run automatically
* ✅ Trigger on events (file arrival, table updates)
* ✅ Chain multiple tasks together
* ✅ Centralized monitoring and alerting
* ✅ Production-ready automation

---

### **Key Benefits:**

* 🔄 **Automation** - Set it and forget it
* ⏰ **Scheduling** - Run on cron schedules
* 🔗 **Orchestration** - Chain multiple tasks
* 📊 **Monitoring** - Track runs and failures
* 🔔 **Notifications** - Alert on success/failure
* 🎛️ **Control** - Manage compute and configs

%undefined
### 🔄 Jobs vs Pipelines: What's the Difference?

**Understanding the relationship:**

---

### **Lakeflow Pipeline (DLT Pipeline):**

**What it is:**
* The **data transformation logic**
* Defines tables, views, and CDC flows
* Written in notebooks using `@dp.table`, `@dp.view`, etc.
* Processes data from source to target

**Example:**
```python
from pyspark import pipelines as dp

@dp.table
def bronze_customers():
    return spark.readStream.table("raw.customers")

@dp.table
def silver_customers():
    return spark.readStream.table("bronze_customers").filter(...)
```

**Think of it as:** The "what" - what data transformations to perform

---

### **Lakeflow Job:**

**What it is:**
* The **orchestration wrapper** around pipelines
* Defines **when** and **how** to run pipelines
* Manages compute, scheduling, and monitoring
* Can run multiple tasks (pipelines, notebooks, etc.)

**Example:**
```python
job = {
    "name": "customer_pipeline_job",
    "tasks": [{
        "task_key": "run_customer_pipeline",
        "pipeline_task": {
            "pipeline_id": "abc-123-def"
        }
    }],
    "schedule": {"quartz_cron_expression": "0 0 * * * ?"}
}
```

**Think of it as:** The "when" and "how" - when to run and how to configure

---

### **Relationship:**

```
┌─────────────────────────────────────┐
│         Lakeflow Job                │
│  (Orchestration & Scheduling)       │
│                                     │
│  ┌───────────────────────────────┐ │
│  │   Task 1: DLT Pipeline        │ │
│  │   (Data Transformations)      │ │
│  └───────────────────────────────┘ │
│                                     │
│  ┌───────────────────────────────┐ │
│  │   Task 2: Notebook            │ │
│  │   (Post-processing)           │ │
│  └───────────────────────────────┘ │
└─────────────────────────────────────┘
```

**Key point:** Jobs **run** pipelines, pipelines **transform** data

%undefined
### 🧩 Job Components

A Lakeflow Job consists of several key components:

---

### **1. Tasks**

**What they are:**
* Individual units of work within a job
* Can be DLT pipelines, notebooks, Python scripts, etc.
* Execute in sequence or parallel

**Types of tasks:**
* **Pipeline Task** - Run a DLT pipeline
* **Notebook Task** - Run a notebook
* **Python Task** - Run Python code
* **SQL Task** - Run SQL queries
* **dbt Task** - Run dbt models

**Example:**
```python
"tasks": [
    {
        "task_key": "ingest_data",
        "pipeline_task": {"pipeline_id": "pipeline-123"}
    },
    {
        "task_key": "send_report",
        "notebook_task": {"notebook_path": "/Reports/daily_summary"}
    }
]
```

---

### **2. Schedule**

**What it is:**
* Defines when the job runs automatically
* Uses cron expressions or simple intervals

**Examples:**
```python
# Run daily at midnight
"schedule": {"quartz_cron_expression": "0 0 0 * * ?"}

# Run every hour
"schedule": {"quartz_cron_expression": "0 0 * * * ?"}

# Run every Monday at 9 AM
"schedule": {"quartz_cron_expression": "0 0 9 ? * MON"}
```

---

### **3. Compute Configuration**

**What it is:**
* Defines the cluster/compute for task execution
* Can use existing clusters or create new ones

**Options:**
* **Existing cluster** - Use a running cluster
* **New cluster** - Create cluster per run
* **Job cluster** - Shared cluster for all tasks
* **Serverless** - Use serverless compute (recommended)

---

### **4. Parameters**

**What they are:**
* Dynamic values passed to tasks at runtime
* Allow reusable jobs with different inputs

**Example:**
```python
"parameters": {
    "start_date": "2026-01-01",
    "environment": "production"
}
```

---

### **5. Notifications**

**What they are:**
* Alerts sent on job success, failure, or start
* Can notify via email, Slack, PagerDuty, etc.

**Example:**
```python
"email_notifications": {
    "on_failure": ["team@company.com"],
    "on_success": ["manager@company.com"]
}
```

%undefined
## Part 2: Creating Jobs Programmatically 🔧

Let's learn how to create and manage jobs using the Databricks Jobs API.

---

%undefined
### 🔌 Databricks Jobs API

**What is it?**
* REST API for managing Databricks jobs
* Create, update, delete, and run jobs programmatically
* Accessible via Python SDK or direct HTTP calls

---

### **Why Use the API?**

**Benefits:**
* ✅ **Infrastructure as Code** - Version control your jobs
* ✅ **Automation** - Create jobs from CI/CD pipelines
* ✅ **Consistency** - Standardize job configurations
* ✅ **Scalability** - Manage hundreds of jobs programmatically
* ✅ **Flexibility** - Dynamic job creation based on logic

---

### **Two Ways to Use the API:**

**1. Databricks SDK (Recommended):**
```python
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()
job = w.jobs.create(
    name="my_pipeline_job",
    tasks=[...]
)
```

**2. Direct REST API:**
```python
import requests

response = requests.post(
    f"{workspace_url}/api/2.1/jobs/create",
    headers={"Authorization": f"Bearer {token}"},
    json={"name": "my_pipeline_job", "tasks": [...]}
)
```

---

**In this demo, we'll use the Databricks SDK for simplicity.**

In [0]:
# Install Databricks SDK
%pip install databricks-sdk --quiet

dbutils.library.restartPython()

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service import jobs
import json

# Initialize the Databricks workspace client
# This automatically uses the notebook's authentication
w = WorkspaceClient()

print("✅ Databricks SDK initialized successfully!")
print(f"📍 Workspace: {w.config.host}")

%undefined
### 🎯 Creating Your First Job

Let's create a simple job that runs a DLT pipeline.

**Steps:**
1. Define the job configuration
2. Create the job using the API
3. Verify the job was created

---

In [0]:
# ============================================
# EXAMPLE 1: Simple Pipeline Job
# ============================================

# First, let's find an existing pipeline to use
# (In practice, you'd use your own pipeline ID)

try:
    # List existing pipelines
    pipelines = w.pipelines.list_pipelines()
    pipeline_list = list(pipelines)
    
    if pipeline_list:
        # Use the first pipeline as an example
        example_pipeline = pipeline_list[0]
        pipeline_id = example_pipeline.pipeline_id
        pipeline_name = example_pipeline.name
        
        print(f"📋 Found pipeline: {pipeline_name}")
        print(f"🆔 Pipeline ID: {pipeline_id}")
        
        # Create a simple job to run this pipeline
        job = w.jobs.create(
            name=f"Demo: {pipeline_name} - Daily Run",
            tasks=[
                jobs.Task(
                    task_key="run_pipeline",
                    description="Run the DLT pipeline",
                    pipeline_task=jobs.PipelineTask(
                        pipeline_id=pipeline_id,
                        full_refresh=False  # Incremental run
                    )
                )
            ],
            # Optional: Add tags for organization
            tags={
                "environment": "demo",
                "created_by": "lakeflow_jobs_demo"
            }
        )
        
        print(f"\n✅ Job created successfully!")
        print(f"🆔 Job ID: {job.job_id}")
        print(f"🔗 View job: {w.config.host}/#job/{job.job_id}")
        
    else:
        print("⚠️ No pipelines found. Please create a pipeline first.")
        print("💡 Tip: Complete the Lakeflow Pipeline Fundamentals demo first.")
        
except Exception as e:
    print(f"❌ Error: {e}")
    print("💡 Make sure you have permissions to create jobs.")

%undefined
### 💡 Understanding the Job Creation Code

**What we did:**

---

### **1. Initialized the Workspace Client:**
```python
w = WorkspaceClient()
```
* Connects to your Databricks workspace
* Uses notebook authentication automatically
* Provides access to all Databricks APIs

---

### **2. Created a Job:**
```python
job = w.jobs.create(
    name="Demo: Pipeline - Daily Run",
    tasks=[...]
)
```
* `name` - Human-readable job name
* `tasks` - List of tasks to execute

---

### **3. Defined a Pipeline Task:**
```python
jobs.Task(
    task_key="run_pipeline",              # Unique identifier
    description="Run the DLT pipeline",   # Human-readable description
    pipeline_task=jobs.PipelineTask(
        pipeline_id=pipeline_id,          # Which pipeline to run
        full_refresh=False                # Incremental vs full refresh
    )
)
```

**Key parameters:**
* `task_key` - Unique identifier for this task (used for dependencies)
* `pipeline_task` - Specifies this is a DLT pipeline task
* `pipeline_id` - The ID of the pipeline to run
* `full_refresh` - `False` for incremental, `True` to reprocess all data

---

### **4. Added Tags:**
```python
tags={
    "environment": "demo",
    "created_by": "lakeflow_jobs_demo"
}
```
* Helps organize and filter jobs
* Useful for cost tracking and governance

%undefined
## Part 3: Job Configuration and Parameters ⚙️

Let's explore advanced job configurations including compute, parameters, and task dependencies.

---

%undefined
### 💻 Compute Configuration Options

**Jobs can use different compute strategies:**

---

### **1. Serverless Compute (Recommended)**

**What it is:**
* Managed compute that scales automatically
* No cluster management required
* Fast startup times
* Pay only for what you use

**When to use:**
* ✅ Most production workloads
* ✅ When you want simplicity
* ✅ Variable workload sizes

**Configuration:**
```python
# For pipeline tasks, serverless is automatic
# No additional configuration needed!
pipeline_task=jobs.PipelineTask(
    pipeline_id=pipeline_id
)
```

---

### **2. Job Cluster**

**What it is:**
* Dedicated cluster created for the job
* Terminated after job completes
* Full control over cluster configuration

**When to use:**
* ✅ Need specific cluster configurations
* ✅ Custom libraries or init scripts
* ✅ Specific instance types

**Configuration:**
```python
job_cluster_key="my_cluster",
job_clusters=[
    jobs.JobCluster(
        job_cluster_key="my_cluster",
        new_cluster=jobs.ClusterSpec(
            spark_version="14.3.x-scala2.12",
            node_type_id="i3.xlarge",
            num_workers=2,
            spark_conf={
                "spark.databricks.delta.preview.enabled": "true"
            }
        )
    )
]
```

---

### **3. Existing Cluster**

**What it is:**
* Use an already-running cluster
* Cluster must be running when job starts

**When to use:**
* ✅ Development and testing
* ✅ Interactive debugging
* ❌ Not recommended for production

**Configuration:**
```python
existing_cluster_id="0123-456789-abcdef"
```

In [0]:
# ============================================
# EXAMPLE 2: Job with Parameters
# ============================================

# Create a job that accepts parameters
# Parameters allow dynamic configuration at runtime

try:
    # Get a pipeline to use
    pipelines = w.pipelines.list_pipelines()
    pipeline_list = list(pipelines)
    
    if pipeline_list:
        pipeline_id = pipeline_list[0].pipeline_id
        
        # Create job with parameters
        job_with_params = w.jobs.create(
            name="Demo: Parameterized Pipeline Job",
            tasks=[
                jobs.Task(
                    task_key="run_pipeline_with_params",
                    description="Run pipeline with custom parameters",
                    pipeline_task=jobs.PipelineTask(
                        pipeline_id=pipeline_id,
                        full_refresh=False
                    )
                )
            ],
            # Define default parameters
            parameters=[
                jobs.JobParameterDefinition(
                    name="environment",
                    default="development"
                ),
                jobs.JobParameterDefinition(
                    name="start_date",
                    default="2026-01-01"
                ),
                jobs.JobParameterDefinition(
                    name="end_date",
                    default="2026-01-31"
                )
            ],
            tags={
                "environment": "demo",
                "type": "parameterized"
            }
        )
        
        print(f"✅ Parameterized job created!")
        print(f"🆔 Job ID: {job_with_params.job_id}")
        print(f"\n📋 Parameters:")
        for param in job_with_params.settings.parameters:
            print(f"   • {param.name}: {param.default}")
        print(f"\n🔗 View job: {w.config.host}/#job/{job_with_params.job_id}")
        
    else:
        print("⚠️ No pipelines found.")
        
except Exception as e:
    print(f"❌ Error: {e}")

In [0]:
# ============================================
# EXAMPLE 3: Multi-Task Job with Dependencies
# ============================================

# Create a job with multiple tasks that run in sequence

try:
    pipelines = w.pipelines.list_pipelines()
    pipeline_list = list(pipelines)
    
    if pipeline_list:
        pipeline_id = pipeline_list[0].pipeline_id
        
        # Create a multi-task job
        multi_task_job = w.jobs.create(
            name="Demo: Multi-Task Pipeline Job",
            tasks=[
                # Task 1: Run the pipeline
                jobs.Task(
                    task_key="ingest_and_transform",
                    description="Run DLT pipeline to ingest and transform data",
                    pipeline_task=jobs.PipelineTask(
                        pipeline_id=pipeline_id,
                        full_refresh=False
                    )
                ),
                # Task 2: Run a notebook (depends on Task 1)
                jobs.Task(
                    task_key="generate_report",
                    description="Generate summary report after pipeline completes",
                    depends_on=[
                        jobs.TaskDependency(task_key="ingest_and_transform")
                    ],
                    notebook_task=jobs.NotebookTask(
                        notebook_path=f"{dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()}",
                        base_parameters={
                            "report_type": "summary",
                            "output_format": "html"
                        }
                    ),
                    # For notebook tasks, we need to specify compute
                    new_cluster=jobs.ClusterSpec(
                        spark_version=w.clusters.select_spark_version(latest=True),
                        node_type_id=w.clusters.select_node_type(local_disk=True, min_memory_gb=8),
                        num_workers=1
                    )
                ),
                # Task 3: Send notification (depends on Task 2)
                jobs.Task(
                    task_key="send_notification",
                    description="Send completion notification",
                    depends_on=[
                        jobs.TaskDependency(task_key="generate_report")
                    ],
                    notebook_task=jobs.NotebookTask(
                        notebook_path=f"{dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()}",
                        base_parameters={
                            "action": "notify",
                            "message": "Pipeline completed successfully"
                        }
                    ),
                    new_cluster=jobs.ClusterSpec(
                        spark_version=w.clusters.select_spark_version(latest=True),
                        node_type_id=w.clusters.select_node_type(local_disk=True, min_memory_gb=8),
                        num_workers=1
                    )
                )
            ],
            tags={
                "environment": "demo",
                "type": "multi_task"
            }
        )
        
        print(f"✅ Multi-task job created!")
        print(f"🆔 Job ID: {multi_task_job.job_id}")
        print(f"\n📋 Task Flow:")
        print(f"   1️⃣ ingest_and_transform (DLT Pipeline)")
        print(f"   2️⃣ generate_report (Notebook) - depends on task 1")
        print(f"   3️⃣ send_notification (Notebook) - depends on task 2")
        print(f"\n🔗 View job: {w.config.host}/#job/{multi_task_job.job_id}")
        
    else:
        print("⚠️ No pipelines found.")
        
except Exception as e:
    print(f"❌ Error: {e}")

%undefined
### 🔗 Task Dependencies

**How task dependencies work:**

---

### **Sequential Execution:**

```python
tasks=[
    jobs.Task(
        task_key="task_1",
        pipeline_task=...
    ),
    jobs.Task(
        task_key="task_2",
        depends_on=[jobs.TaskDependency(task_key="task_1")],
        notebook_task=...
    ),
    jobs.Task(
        task_key="task_3",
        depends_on=[jobs.TaskDependency(task_key="task_2")],
        notebook_task=...
    )
]
```

**Execution order:**
```
task_1 → task_2 → task_3
```

---

### **Parallel Execution:**

```python
tasks=[
    jobs.Task(
        task_key="task_1",
        pipeline_task=...
    ),
    jobs.Task(
        task_key="task_2a",
        depends_on=[jobs.TaskDependency(task_key="task_1")],
        notebook_task=...
    ),
    jobs.Task(
        task_key="task_2b",
        depends_on=[jobs.TaskDependency(task_key="task_1")],
        notebook_task=...
    ),
    jobs.Task(
        task_key="task_3",
        depends_on=[
            jobs.TaskDependency(task_key="task_2a"),
            jobs.TaskDependency(task_key="task_2b")
        ],
        notebook_task=...
    )
]
```

**Execution order:**
```
        task_1
       /      \
   task_2a  task_2b  (run in parallel)
       \      /
        task_3
```

---

### **Key Points:**

* Tasks without dependencies run immediately
* Tasks with dependencies wait for parent tasks to complete
* Multiple tasks can depend on the same parent (parallel execution)
* A task can depend on multiple parents (waits for all to complete)

%undefined
## Part 4: Scheduling and Triggers ⏰

Let's learn how to schedule jobs to run automatically.

---

%undefined
### 📅 Job Scheduling Options

**Databricks Jobs support multiple scheduling methods:**

---

### **1. Cron Schedule**

**What it is:**
* Time-based scheduling using cron expressions
* Most common scheduling method
* Supports complex schedules

**Cron format (Quartz):**
```
┌───────────── second (0-59)
│ ┌───────────── minute (0-59)
│ │ ┌───────────── hour (0-23)
│ │ │ ┌───────────── day of month (1-31)
│ │ │ │ ┌───────────── month (1-12 or JAN-DEC)
│ │ │ │ │ ┌───────────── day of week (0-6 or SUN-SAT)
│ │ │ │ │ │
* * * * * *
```

**Common examples:**
```python
# Every day at midnight
"0 0 0 * * ?"

# Every hour
"0 0 * * * ?"

# Every 15 minutes
"0 */15 * * * ?"

# Every Monday at 9 AM
"0 0 9 ? * MON"

# Every weekday at 6 AM
"0 0 6 ? * MON-FRI"

# First day of every month at midnight
"0 0 0 1 * ?"
```

---

### **2. Continuous (Always Running)**

**What it is:**
* Job runs continuously
* Restarts automatically after completion
* Useful for streaming pipelines

**When to use:**
* ✅ Real-time data processing
* ✅ Continuous streaming pipelines
* ❌ Not for batch processing

---

### **3. File Arrival Trigger**

**What it is:**
* Job runs when new files arrive in a location
* Monitors cloud storage paths
* Event-driven processing

**When to use:**
* ✅ Process files as they arrive
* ✅ Event-driven workflows
* ✅ Unpredictable data arrival

---

### **4. Manual (On-Demand)**

**What it is:**
* No automatic schedule
* Run manually via UI or API

**When to use:**
* ✅ Ad-hoc processing
* ✅ Testing and development
* ✅ Triggered by external systems

In [0]:
# ============================================
# EXAMPLE 4: Scheduled Job (Daily at Midnight)
# ============================================

try:
    pipelines = w.pipelines.list_pipelines()
    pipeline_list = list(pipelines)
    
    if pipeline_list:
        pipeline_id = pipeline_list[0].pipeline_id
        
        # Create a scheduled job
        scheduled_job = w.jobs.create(
            name="Demo: Daily Pipeline Job (Midnight)",
            tasks=[
                jobs.Task(
                    task_key="daily_pipeline_run",
                    description="Run pipeline daily at midnight",
                    pipeline_task=jobs.PipelineTask(
                        pipeline_id=pipeline_id,
                        full_refresh=False
                    )
                )
            ],
            # Schedule: Every day at midnight (UTC)
            schedule=jobs.CronSchedule(
                quartz_cron_expression="0 0 0 * * ?",
                timezone_id="UTC",
                pause_status=jobs.PauseStatus.UNPAUSED
            ),
            tags={
                "environment": "demo",
                "schedule": "daily"
            }
        )
        
        print(f"✅ Scheduled job created!")
        print(f"🆔 Job ID: {scheduled_job.job_id}")
        print(f"⏰ Schedule: Daily at midnight (UTC)")
        print(f"📅 Cron: {scheduled_job.settings.schedule.quartz_cron_expression}")
        print(f"🔗 View job: {w.config.host}/#job/{scheduled_job.job_id}")
        
    else:
        print("⚠️ No pipelines found.")
        
except Exception as e:
    print(f"❌ Error: {e}")

In [0]:
# ============================================
# EXAMPLE 5: File Arrival Trigger
# ============================================

# Note: File arrival triggers require specific permissions
# and cloud storage configuration

try:
    pipelines = w.pipelines.list_pipelines()
    pipeline_list = list(pipelines)
    
    if pipeline_list:
        pipeline_id = pipeline_list[0].pipeline_id
        
        # Create a file-triggered job
        file_trigger_job = w.jobs.create(
            name="Demo: File Arrival Triggered Job",
            tasks=[
                jobs.Task(
                    task_key="process_new_files",
                    description="Process pipeline when new files arrive",
                    pipeline_task=jobs.PipelineTask(
                        pipeline_id=pipeline_id,
                        full_refresh=False
                    )
                )
            ],
            # Trigger on file arrival
            trigger=jobs.TriggerSettings(
                file_arrival=jobs.FileArrivalTriggerConfiguration(
                    url="s3://my-bucket/incoming-data/",  # Change to your path
                    min_time_between_triggers_seconds=60,  # Wait 60s between triggers
                    wait_after_last_change_seconds=30      # Wait 30s after last file
                )
            ),
            tags={
                "environment": "demo",
                "trigger": "file_arrival"
            }
        )
        
        print(f"✅ File-triggered job created!")
        print(f"🆔 Job ID: {file_trigger_job.job_id}")
        print(f"📁 Monitoring: s3://my-bucket/incoming-data/")
        print(f"⏱️ Min time between triggers: 60 seconds")
        print(f"🔗 View job: {w.config.host}/#job/{file_trigger_job.job_id}")
        print(f"\n💡 Note: Update the S3 path to your actual data location")
        
    else:
        print("⚠️ No pipelines found.")
        
except Exception as e:
    print(f"❌ Error: {e}")
    print(f"💡 File arrival triggers require cloud storage permissions")

%undefined
### 🎯 Challenge 1: Create a Scheduled Job

**Your task:**

Create a job that runs a pipeline every Monday at 9 AM.

---

**Requirements:**

1. **Job name:** "Challenge 1: Weekly Pipeline Job"
2. **Task:**
   * Task key: "weekly_pipeline_run"
   * Use any existing pipeline
   * Incremental refresh (not full refresh)
3. **Schedule:**
   * Every Monday at 9:00 AM UTC
   * Cron expression: `"0 0 9 ? * MON"`
   * Timezone: "UTC"
   * Status: UNPAUSED
4. **Tags:**
   * environment: "challenge"
   * schedule: "weekly"

---

**Write your code below:** 👇

In [0]:
# ============================================
# CHALLENGE 1: Your solution here
# ============================================

# TODO: Create a job that runs every Monday at 9 AM



%undefined
## Part 5: Monitoring and Notifications 📊

Let's learn how to monitor job execution and set up notifications.

---

%undefined
### 📊 Monitoring Job Execution

**Ways to monitor jobs:**

---

### **1. Databricks UI**

**Workflows Page:**
* View all jobs and their status
* See run history and duration
* Access logs and error messages
* Monitor active runs

**Navigation:**
```
Workflows → Jobs → [Your Job] → Runs
```

---

### **2. Jobs API**

**List job runs:**
```python
# Get recent runs for a job
runs = w.jobs.list_runs(job_id=job_id, limit=10)

for run in runs:
    print(f"Run {run.run_id}: {run.state.life_cycle_state}")
```

**Get run details:**
```python
# Get details of a specific run
run = w.jobs.get_run(run_id=run_id)

print(f"Status: {run.state.life_cycle_state}")
print(f"Result: {run.state.result_state}")
print(f"Start: {run.start_time}")
print(f"End: {run.end_time}")
```

---

### **3. Run States**

**Life cycle states:**
* `PENDING` - Waiting to start
* `RUNNING` - Currently executing
* `TERMINATING` - Shutting down
* `TERMINATED` - Completed
* `SKIPPED` - Skipped due to conditions
* `INTERNAL_ERROR` - System error

**Result states (for terminated runs):**
* `SUCCESS` - Completed successfully
* `FAILED` - Failed with errors
* `TIMEDOUT` - Exceeded timeout
* `CANCELED` - Manually canceled

---

### **4. Metrics to Monitor**

**Key metrics:**
* ✅ Success rate
* ✅ Run duration
* ✅ Failure frequency
* ✅ Queue time
* ✅ Cost per run

In [0]:
# ============================================
# EXAMPLE 6: List and Monitor Jobs
# ============================================

try:
    # List all jobs (limit to 10 for demo)
    all_jobs = w.jobs.list(limit=10)
    
    print("📋 Your Jobs:\n")
    
    for job in all_jobs:
        print(f"🆔 Job ID: {job.job_id}")
        print(f"📝 Name: {job.settings.name}")
        
        # Get recent runs for this job
        runs = list(w.jobs.list_runs(job_id=job.job_id, limit=3))
        
        if runs:
            print(f"📊 Recent Runs:")
            for run in runs:
                status = run.state.life_cycle_state
                result = run.state.result_state if run.state.result_state else "N/A"
                print(f"   • Run {run.run_id}: {status} - {result}")
        else:
            print(f"📊 No runs yet")
        
        print(f"🔗 View: {w.config.host}/#job/{job.job_id}")
        print("-" * 60)
        
except Exception as e:
    print(f"❌ Error: {e}")

%undefined
### 🔔 Notification Configuration

**Set up alerts for job events:**

---

### **Email Notifications**

**Basic email alerts:**
```python
email_notifications=jobs.JobEmailNotifications(
    on_start=["team@company.com"],
    on_success=["manager@company.com"],
    on_failure=["oncall@company.com", "team@company.com"],
    no_alert_for_skipped_runs=True
)
```

**When to notify:**
* `on_start` - Job starts running
* `on_success` - Job completes successfully
* `on_failure` - Job fails
* `on_duration_warning_threshold_exceeded` - Job runs longer than expected

---

### **Webhook Notifications**

**Integrate with external systems:**
```python
webhook_notifications=jobs.WebhookNotifications(
    on_start=[jobs.Webhook(id="webhook-123")],
    on_success=[jobs.Webhook(id="webhook-456")],
    on_failure=[jobs.Webhook(id="webhook-789")]
)
```

**Use cases:**
* Slack notifications
* PagerDuty alerts
* Custom monitoring systems
* Ticketing systems (Jira, ServiceNow)

---

### **Notification Best Practices**

**Do:**
* ✅ Always notify on failures
* ✅ Use different recipients for different events
* ✅ Set up escalation for critical jobs
* ✅ Include job context in notifications

**Don't:**
* ❌ Spam with success notifications for every job
* ❌ Use personal emails for production alerts
* ❌ Ignore notification fatigue

In [0]:
# ============================================
# EXAMPLE 7: Job with Email Notifications
# ============================================

try:
    pipelines = w.pipelines.list_pipelines()
    pipeline_list = list(pipelines)
    
    if pipeline_list:
        pipeline_id = pipeline_list[0].pipeline_id
        
        # Get current user email
        current_user = w.current_user.me()
        user_email = current_user.user_name
        
        # Create job with notifications
        notified_job = w.jobs.create(
            name="Demo: Job with Notifications",
            tasks=[
                jobs.Task(
                    task_key="pipeline_with_alerts",
                    description="Pipeline with email notifications",
                    pipeline_task=jobs.PipelineTask(
                        pipeline_id=pipeline_id,
                        full_refresh=False
                    )
                )
            ],
            # Email notifications
            email_notifications=jobs.JobEmailNotifications(
                on_failure=[user_email],  # Notify on failure
                on_success=[user_email],  # Notify on success
                no_alert_for_skipped_runs=True
            ),
            # Timeout settings
            timeout_seconds=3600,  # 1 hour timeout
            tags={
                "environment": "demo",
                "notifications": "enabled"
            }
        )
        
        print(f"✅ Job with notifications created!")
        print(f"🆔 Job ID: {notified_job.job_id}")
        print(f"📧 Notifications sent to: {user_email}")
        print(f"⏱️ Timeout: 1 hour")
        print(f"🔗 View job: {w.config.host}/#job/{notified_job.job_id}")
        
    else:
        print("⚠️ No pipelines found.")
        
except Exception as e:
    print(f"❌ Error: {e}")

%undefined
### ▶️ Running Jobs Programmatically

**Trigger job runs via API:**

---

### **Run Now (Manual Trigger)**

**Start a job immediately:**
```python
# Run a job now
run = w.jobs.run_now(job_id=job_id)

print(f"Started run: {run.run_id}")
```

**Run with custom parameters:**
```python
# Run with different parameters
run = w.jobs.run_now(
    job_id=job_id,
    job_parameters={
        "environment": "production",
        "start_date": "2026-01-26"
    }
)
```

---

### **Wait for Completion**

**Block until run completes:**
```python
# Start run and wait for completion
run = w.jobs.run_now(job_id=job_id)

# Wait for run to complete (blocks)
result = w.jobs.wait_get_run_job_terminated_or_skipped(run_id=run.run_id)

if result.state.result_state == jobs.RunResultState.SUCCESS:
    print("✅ Job completed successfully!")
else:
    print(f"❌ Job failed: {result.state.state_message}")
```

---

### **Cancel a Run**

**Stop a running job:**
```python
# Cancel a specific run
w.jobs.cancel_run(run_id=run_id)

print(f"Canceled run: {run_id}")
```

In [0]:
# ============================================
# EXAMPLE 8: Run a Job and Monitor
# ============================================

try:
    # Get the first demo job we created
    all_jobs = list(w.jobs.list(limit=5))
    
    if all_jobs:
        # Find a job created by this demo
        demo_job = None
        for job in all_jobs:
            if "Demo:" in job.settings.name:
                demo_job = job
                break
        
        if demo_job:
            print(f"🎯 Running job: {demo_job.settings.name}")
            print(f"🆔 Job ID: {demo_job.job_id}")
            
            # Start the job
            run = w.jobs.run_now(job_id=demo_job.job_id)
            
            print(f"\n▶️ Started run: {run.run_id}")
            print(f"🔗 View run: {w.config.host}/#job/{demo_job.job_id}/run/{run.run_id}")
            print(f"\n⏳ Job is running... (not waiting for completion in demo)")
            print(f"💡 Check the link above to monitor progress")
            
            # Note: We're not waiting for completion to avoid long delays
            # In production, you might use:
            # result = w.jobs.wait_get_run_job_terminated_or_skipped(run_id=run.run_id)
            
        else:
            print("⚠️ No demo jobs found. Create a job first.")
    else:
        print("⚠️ No jobs found.")
        
except Exception as e:
    print(f"❌ Error: {e}")

%undefined
### 🎯 Challenge 2: Create a Job with Notifications

**Your task:**

Create a job with email notifications and run it.

---

**Requirements:**

1. **Job name:** "Challenge 2: Monitored Pipeline Job"
2. **Task:**
   * Task key: "monitored_pipeline"
   * Use any existing pipeline
   * Incremental refresh
3. **Notifications:**
   * Send email to your user email on failure
   * Send email to your user email on success
   * No alerts for skipped runs
4. **Timeout:** 30 minutes (1800 seconds)
5. **Tags:**
   * environment: "challenge"
   * monitoring: "enabled"
6. **After creating the job, run it using `w.jobs.run_now()`**

---

**Hints:**
* Use `w.current_user.me().user_name` to get your email
* Use `jobs.JobEmailNotifications()` for notifications
* Use `timeout_seconds` parameter

---

**Write your code below:** 👇

In [0]:
# ============================================
# CHALLENGE 2: Your solution here
# ============================================

# TODO: Create a job with notifications and run it



%undefined
## Part 6: Best Practices and Patterns ✅

Let's cover best practices for production Lakeflow Jobs.

---

%undefined
### 🎯 Lakeflow Jobs Best Practices

---

### **1. Naming Conventions**

**Use descriptive, consistent names:**

```python
# Good
"Production: Customer Pipeline - Daily"
"Dev: Sales Analytics - Hourly"
"Staging: Inventory Sync - Real-time"

# Bad
"job1"
"test"
"my_pipeline"
```

**Include:**
* Environment (Production, Dev, Staging)
* Purpose (what it does)
* Frequency (Daily, Hourly, etc.)

---

### **2. Use Tags for Organization**

**Tag everything:**

```python
tags={
    "environment": "production",
    "team": "data-engineering",
    "cost_center": "analytics",
    "criticality": "high",
    "owner": "data-team@company.com"
}
```

**Benefits:**
* Easy filtering and searching
* Cost allocation
* Ownership tracking
* Governance and compliance

---

### **3. Compute Strategy**

**Choose the right compute:**

**For DLT Pipelines:**
* ✅ Use serverless (default)
* ✅ Let DLT manage compute
* ❌ Don't override unless necessary

**For Notebook Tasks:**
* ✅ Use job clusters (created per run)
* ✅ Right-size for workload
* ❌ Avoid existing clusters in production

---

### **4. Error Handling**

**Plan for failures:**

```python
# Set retries
max_retries=3,
min_retry_interval_millis=60000,  # 1 minute

# Set timeouts
timeout_seconds=7200,  # 2 hours

# Configure notifications
email_notifications=jobs.JobEmailNotifications(
    on_failure=["oncall@company.com"]
)
```

**Best practices:**
* ✅ Set appropriate timeouts
* ✅ Configure retries for transient failures
* ✅ Always notify on failures
* ✅ Log errors for debugging

---

### **5. Scheduling Strategy**

**Choose the right schedule:**

**Batch Processing:**
* ✅ Use cron schedules
* ✅ Run during off-peak hours
* ✅ Consider data freshness requirements

**Streaming/Real-time:**
* ✅ Use continuous mode
* ✅ Monitor for backlog
* ✅ Set up alerting

**Event-driven:**
* ✅ Use file arrival triggers
* ✅ Set minimum time between triggers
* ✅ Handle duplicate events

---

### **6. Monitoring and Alerting**

**Monitor everything:**

```python
# Critical jobs
email_notifications=jobs.JobEmailNotifications(
    on_failure=["oncall@company.com", "team@company.com"],
    on_duration_warning_threshold_exceeded=["manager@company.com"]
)

# Set SLAs
timeout_seconds=3600,  # Fail if exceeds 1 hour
```

**What to monitor:**
* ✅ Success/failure rate
* ✅ Run duration trends
* ✅ Cost per run
* ✅ Data quality metrics
* ✅ Pipeline backlog

---

### **7. Parameters and Configuration**

**Make jobs reusable:**

```python
# Define parameters
parameters=[
    jobs.JobParameterDefinition(
        name="environment",
        default="production"
    ),
    jobs.JobParameterDefinition(
        name="date",
        default="{{job.start_time.iso_date}}"
    )
]
```

**Benefits:**
* ✅ Reuse same job for different environments
* ✅ Override values at runtime
* ✅ Easier testing and debugging

---

### **8. Documentation**

**Document your jobs:**

```python
jobs.Task(
    task_key="ingest_customers",
    description="Ingest customer data from S3, apply CDC, and update silver table. Runs daily at midnight. Owner: data-team@company.com",
    pipeline_task=...
)
```

**Include:**
* Purpose and business context
* Data sources and targets
* Schedule and frequency
* Owner and contact info
* Dependencies and prerequisites

---

### **9. Testing Strategy**

**Test before production:**

1. **Development:**
   * Create dev version of job
   * Use dev pipelines and data
   * Test manually

2. **Staging:**
   * Create staging version
   * Use production-like data
   * Test scheduling

3. **Production:**
   * Deploy with monitoring
   * Start with manual runs
   * Enable schedule after validation

---

### **10. Version Control**

**Treat jobs as code:**

```python
# Store job definitions in Git
# Use CI/CD to deploy
# Track changes over time
```

**Benefits:**
* ✅ Audit trail of changes
* ✅ Easy rollback
* ✅ Peer review
* ✅ Automated deployment

In [0]:
# ============================================
# EXAMPLE 9: Production-Ready Job
# ============================================

# This example shows a complete production-ready job
# with all best practices applied

try:
    pipelines = w.pipelines.list_pipelines()
    pipeline_list = list(pipelines)
    
    if pipeline_list:
        pipeline_id = pipeline_list[0].pipeline_id
        current_user = w.current_user.me()
        user_email = current_user.user_name
        
        # Create production-ready job
        prod_job = w.jobs.create(
            name="Production: Customer Data Pipeline - Daily",
            
            # Tasks with clear descriptions
            tasks=[
                jobs.Task(
                    task_key="ingest_and_transform",
                    description="Ingest customer data from source systems, apply transformations and data quality checks. Outputs to silver.customers table.",
                    pipeline_task=jobs.PipelineTask(
                        pipeline_id=pipeline_id,
                        full_refresh=False
                    ),
                    timeout_seconds=3600,  # 1 hour timeout per task
                )
            ],
            
            # Schedule: Daily at 2 AM UTC (off-peak)
            schedule=jobs.CronSchedule(
                quartz_cron_expression="0 0 2 * * ?",
                timezone_id="UTC",
                pause_status=jobs.PauseStatus.UNPAUSED
            ),
            
            # Email notifications
            email_notifications=jobs.JobEmailNotifications(
                on_failure=[user_email],  # Alert on failure
                on_success=[],  # Don't spam on success
                no_alert_for_skipped_runs=True
            ),
            
            # Retry configuration
            max_retries=2,
            min_retry_interval_millis=300000,  # 5 minutes between retries
            retry_on_timeout=True,
            
            # Overall job timeout
            timeout_seconds=7200,  # 2 hours max
            
            # Parameters for flexibility
            parameters=[
                jobs.JobParameterDefinition(
                    name="environment",
                    default="production"
                ),
                jobs.JobParameterDefinition(
                    name="run_date",
                    default="{{job.start_time.iso_date}}"
                )
            ],
            
            # Tags for organization
            tags={
                "environment": "production",
                "team": "data-engineering",
                "pipeline": "customer-data",
                "criticality": "high",
                "owner": user_email,
                "cost_center": "analytics",
                "schedule": "daily"
            }
        )
        
        print(f"✅ Production-ready job created!")
        print(f"🆔 Job ID: {prod_job.job_id}")
        print(f"\n📋 Configuration:")
        print(f"   • Schedule: Daily at 2 AM UTC")
        print(f"   • Timeout: 2 hours")
        print(f"   • Retries: 2 (with 5 min intervals)")
        print(f"   • Notifications: On failure to {user_email}")
        print(f"   • Tags: {len(prod_job.settings.tags)} tags for organization")
        print(f"\n🔗 View job: {w.config.host}/#job/{prod_job.job_id}")
        
    else:
        print("⚠️ No pipelines found.")
        
except Exception as e:
    print(f"❌ Error: {e}")

%undefined
### 🛠️ Job Management Operations

**Common operations for managing jobs:**

---

### **Update a Job**

```python
# Update job configuration
w.jobs.update(
    job_id=job_id,
    new_settings=jobs.JobSettings(
        name="Updated Job Name",
        schedule=jobs.CronSchedule(
            quartz_cron_expression="0 0 3 * * ?",  # Change to 3 AM
            timezone_id="UTC"
        )
    )
)
```

---

### **Pause/Resume Schedule**

```python
# Pause a scheduled job
w.jobs.update(
    job_id=job_id,
    new_settings=jobs.JobSettings(
        schedule=jobs.CronSchedule(
            quartz_cron_expression="0 0 2 * * ?",
            timezone_id="UTC",
            pause_status=jobs.PauseStatus.PAUSED  # Pause
        )
    )
)

# Resume
w.jobs.update(
    job_id=job_id,
    new_settings=jobs.JobSettings(
        schedule=jobs.CronSchedule(
            quartz_cron_expression="0 0 2 * * ?",
            timezone_id="UTC",
            pause_status=jobs.PauseStatus.UNPAUSED  # Resume
        )
    )
)
```

---

### **Delete a Job**

```python
# Delete a job (careful!)
w.jobs.delete(job_id=job_id)

print(f"Deleted job {job_id}")
```

---

### **Export Job Configuration**

```python
# Get job details
job = w.jobs.get(job_id=job_id)

# Export to JSON for version control
import json

job_config = {
    "name": job.settings.name,
    "tasks": [task.as_dict() for task in job.settings.tasks],
    "schedule": job.settings.schedule.as_dict() if job.settings.schedule else None,
    "tags": job.settings.tags
}

with open(f"job_{job_id}.json", "w") as f:
    json.dump(job_config, f, indent=2)

print(f"Exported job configuration to job_{job_id}.json")
```

In [0]:
# ============================================
# CLEANUP: Delete Demo Jobs (Optional)
# ============================================

# Uncomment and run this cell to clean up demo jobs

# try:
#     # List all jobs
#     all_jobs = list(w.jobs.list())
#     
#     # Find and delete demo jobs
#     deleted_count = 0
#     for job in all_jobs:
#         if "Demo:" in job.settings.name or "Challenge" in job.settings.name:
#             print(f"🗑️ Deleting: {job.settings.name} (ID: {job.job_id})")
#             w.jobs.delete(job_id=job.job_id)
#             deleted_count += 1
#     
#     print(f"\n✅ Deleted {deleted_count} demo jobs")
#     
# except Exception as e:
#     print(f"❌ Error: {e}")

print("💡 Uncomment the code above to delete demo jobs")

%undefined
## Summary: Lakeflow Jobs Mastery 🎓

Congratulations! You've learned how to create and manage Lakeflow Jobs.

---

### **Key Concepts:**

**1. Jobs vs Pipelines:**
* ✅ Pipelines = data transformations (the "what")
* ✅ Jobs = orchestration (the "when" and "how")
* ✅ Jobs run pipelines on schedules or triggers

**2. Job Components:**
* ✅ **Tasks** - Units of work (pipelines, notebooks, etc.)
* ✅ **Schedule** - When to run (cron, continuous, file arrival)
* ✅ **Compute** - Resources for execution
* ✅ **Parameters** - Dynamic configuration
* ✅ **Notifications** - Alerts on events

**3. Databricks SDK:**
* ✅ `WorkspaceClient()` - Initialize client
* ✅ `w.jobs.create()` - Create jobs
* ✅ `w.jobs.run_now()` - Trigger runs
* ✅ `w.jobs.list_runs()` - Monitor execution
* ✅ `w.jobs.update()` - Modify jobs
* ✅ `w.jobs.delete()` - Remove jobs

**4. Task Types:**
* ✅ **Pipeline Task** - Run DLT pipelines
* ✅ **Notebook Task** - Run notebooks
* ✅ **Python Task** - Run Python code
* ✅ **SQL Task** - Run SQL queries

**5. Scheduling:**
* ✅ **Cron** - Time-based (daily, hourly, etc.)
* ✅ **Continuous** - Always running
* ✅ **File Arrival** - Event-driven
* ✅ **Manual** - On-demand

---

### **Best Practices:**

* ✅ Use descriptive naming conventions
* ✅ Tag everything for organization
* ✅ Use serverless compute for DLT
* ✅ Configure retries and timeouts
* ✅ Set up notifications for failures
* ✅ Monitor success rates and duration
* ✅ Make jobs reusable with parameters
* ✅ Document purpose and ownership
* ✅ Test in dev/staging before production
* ✅ Version control job configurations

---

### **Next Steps:**

1. **Create production jobs:**
   * Identify pipelines to automate
   * Define schedules and SLAs
   * Configure monitoring and alerts

2. **Implement CI/CD:**
   * Store job configs in Git
   * Automate deployment
   * Use environments (dev/staging/prod)

3. **Monitor and optimize:**
   * Track job metrics
   * Optimize schedules
   * Reduce costs

---

### **Resources:**

* [Lakeflow Pipeline Fundamentals](#notebook/2846436383063456)
* [Lakeflow Expectations](#notebook/2846436383063443)
* [Lakeflow Auto CDC](#notebook/2846436383063462)
* [Databricks Jobs API Documentation](https://docs.databricks.com/api/workspace/jobs)
* [Databricks SDK Documentation](https://databricks-sdk-py.readthedocs.io/)

---

**Happy orchestrating!** 🚀

%undefined
## Next Steps: Putting It All Together 🎯

**You now have all the skills to build production data pipelines!**

---

### **Complete Workflow:**

**1. Design Pipeline (Lakeflow Fundamentals)**
* Define bronze, silver, gold layers
* Create streaming tables and views
* Implement transformations

**2. Add Data Quality (Lakeflow Expectations)**
* Add expectations for validation
* Use expect, expect_or_drop, expect_or_fail
* Monitor data quality metrics

**3. Implement CDC (Lakeflow Auto CDC)**
* Use Auto CDC for change tracking
* Choose SCD Type 1 or Type 2
* Handle inserts, updates, deletes

**4. Orchestrate (Lakeflow Jobs)**
* Create jobs to run pipelines
* Schedule with cron or triggers
* Configure monitoring and alerts

---

### **Example Production Architecture:**

```
┌─────────────────────────────────────────────────────┐
│                  Lakeflow Job                       │
│              (Scheduled Daily at 2 AM)              │
│                                                     │
│  ┌───────────────────────────────────────────────┐ │
│  │         DLT Pipeline (Bronze → Gold)          │ │
│  │                                               │ │
│  │  Bronze: Ingest with Auto Loader             │ │
│  │     ↓                                         │ │
│  │  Silver: Auto CDC + Expectations             │ │
│  │     ↓                                         │ │
│  │  Gold: Aggregations + Business Logic         │ │
│  └───────────────────────────────────────────────┘ │
│                                                     │
│  ┌───────────────────────────────────────────────┐ │
│  │      Post-Processing Notebook                 │ │
│  │  (Generate reports, update dashboards)        │ │
│  └───────────────────────────────────────────────┘ │
│                                                     │
│  📧 Email notifications on failure                 │
│  🔄 Retry 2x on transient errors                   │
│  ⏱️ 2 hour timeout                                 │
└─────────────────────────────────────────────────────┘
```

---

### **Ready to Build?**

**Start with:**
1. Identify a use case
2. Design your pipeline
3. Implement with best practices
4. Test thoroughly
5. Deploy to production
6. Monitor and iterate

---

**You've got this!** 💪